# ECE/CS 434 | MP3: IMU PDR
<br />
<nav>
    <span class="alert alert-block alert-warning">Due at 11:59PM March 20th 2023 on Gradescope</span> |
    <a href="https://www.gradescope.com/courses/490534/">Gradescope</a> | 
    <a href="https://courses.grainger.illinois.edu/cs434/sp2023/">Course Website</a> | 
</nav><br> 

**Name:** Saif Ur Rahman<br>
**NetID:** saifu2

<hr />  

## Objective
In this MP, you will:
- Implement a step estimation algorithm using accelerometer data.
- Apply signal processing and linear algebra functions such as low/high pass filtering, peak detection, fft, etc. to the step estimation algorithm. 
- Track phone orientation through a sequence of gyroscope data by performing integration.
- Implement simple PDR algorithm: find the final position of the user using steps, orientations and initial position

---
## Problem Overview
In pedestrian dead-reckoning applications, two pieces of information need to be tracked: how far a user walked, and the direction of the walk. In the first part of this MP, you will derive the final orientation using a sequence of gyroscope data. In the second part, you will write a step estimation algorithm using accelerometer data as input and use it to calculate the final position of the user. 

---
## Imports & Setup

### Installing requirements correctly

First. we will make sure that the correct versions of required modules are installed. This ensures that your local Python environment is consistent with the one running on the Gradescope autograder. Just convert the following cell to code and run:

<div class="alert alert-block alert-info"><b>Note:</b> It's preferred that your local environment matches the autograder to prevent possible inconsistencies. However, if you're running into annoying Python version issues but haven't had any issues getting consistent results on the autograder, there is no need to stress over it. Just skip for now and come back when you do encounter inconsistencies:) Ditto below.
</div>

### Your imports
Write your import statements below. If Gradescope reports an error and you believe it is due to an unsupported import, check with the TA to see if it could be added.

In [ ]:
# # Only Google Colab settings, uncomment this if you have to use google drive

# from google.colab import drive
# drive.mount('/content/drive')
# %cd "/content/drive/MyDrive/CS 434 Real-World Algorithms for IoT and Data Science/MP3_PDR"
# !pip install numpy==1.21.3
# !pip install matplotlib==3.4.3
# !pip install pandas==1.4.1

In [35]:
import numpy as np
import pandas as pd
from pathlib import Path
import math
import matplotlib.pyplot as plt
import scipy

# This function is used to format test results. You don't need to touch it.
def display_table(data):
    from IPython.display import HTML, display

    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>{}</h4><td>".format(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### Sanity-check

Running the following code block verifies that the correct module versions are indeed being used. 

Try restarting the Python kernel (or Jupyter) if there is a mismatch even after intalling the correct version. This might happen because Python's `import` statement does not reload already-loaded modules even if they are updated.

In [36]:
if __name__ == '__main__':
    from IPython.display import display, HTML

    def printc(text, color):
        display(HTML("<text style='color:{};weight:700;'>{}</text>".format(color, text)))

    _requirements = [r.split("==") for r in open(
        "requirements.txt", "r").read().strip().split("\n")]

    import sys
    for (module, expected_version) in _requirements:
        try:
            if sys.modules[module].__version__ != expected_version:
                printc("[✕] {} version should to be {}, but {} is installed.".format(
                    module, expected_version, sys.modules[module].__version__), "#f44336")
            else:
                printc("[✓] {} version {} is correct.".format(
                    module, expected_version), "#4caf50")
        except:
            printc("[–] {} is not imported, skipping version check.".format(
                module), "#03a9f4")

---
## Part 1. 3D Orientation Tracking

Assume that phone is static and phone's local coordinate frame and global/world coordinate frame are aligned at the initial moment. Perform gyro integration for each timestamp onward to calculate the phone's orientation. We will provide you with a trace of gyroscope data in CSV format. There are three columns in the file, representing the gyroscope readings in three **local** axes (unit: $rad / s$). The gyroscope is sampled at 100Hz. Your task is to track the phone’s 3D orientation and **output the end direction in which the phone’s local X axis is pointing at in the global frame**.

One way of solving this problem can be:
<ol type="A">
    <li> Assume the gyroscope’s sample interval is $\Delta t$. </li>
    <li> Get the phone's instant rotation axis and rotation angle in the local frame $(\vec{l}, \Delta \theta)$ for each time stamp $t_{i},$ where $\vec{l}=\left(\omega_{x}, \omega_{v}, \omega_{z}\right)$ and $\Delta \theta=\sqrt{\left(\omega_{x}^{2}+\omega_{v}^{2}+\omega_{z}^{2}\right)} \cdot \Delta t$ </li>
    <li> Project the instant rotation axis $\vec{l}$ into the global frame using the phone's $3 \mathrm{D}$ orientation matrix $R_{i}$ at time $t_{i}$. </li>
    <li> Convert the instant rotation axis and angle in global frame into the form of rotation matrix $\Delta R_{i}$. </li>
    <li> Find the total 3D rotation matrix for time $t_{i+1}: R_{i+1}=\Delta R_{i} \cdot R_{i}$ </li>
</ol>

--- 
**Implement the above algorithm in `track_orientation(gyro_file)` below.** 

Do <b>NOT</b> change the function signature. You are, however, free to define and use helper functions.

In [37]:
# This function takes 1 argument:
#     - gyro_file (string) - name of file containing a sequence of gyroscope data
# It returns an array representing the final global direction
# in which the phone's local X axis is pointing.
def track_orientation(gyro_file):
    # Your implementation starts here:
    del_t = 1/100
    R = np.eye(3)
    gyro_pd = pd.read_csv(gyro_file, names=['x', 'y', 'z']) #read csv file
    num_samples = len(gyro_pd['x'])

    for i in range(num_samples):
      # part b
      l = np.array([gyro_pd['x'][i], gyro_pd['y'][i], gyro_pd['z'][i]])
      dtheta = np.sqrt(np.square(l[0])+np.square(l[1])+np.square(l[2])) * del_t

      # part c
      l_global = R@l

      # part d
      c = np.cos(dtheta)
      s = np.sin(dtheta)
      l = l_global / np.linalg.norm(l_global)
      R_delta = np.array([[c + (1 - c) * l[0] ** 2, (1 - c) * l[0] * l[1] - s * l[2], (1 - c) * l[0] * l[2] + s * l[1]],
                        [(1 - c) * l[0] * l[1] + s * l[2], c + (1 - c) * l[1] ** 2, (1 - c) * l[1] * l[2] - s * l[0]],
                        [(1 - c) * l[0] * l[2] - s * l[1], (1 - c) * l[1] * l[2] + s * l[0], c + (1 - c) * l[2] ** 2]])
      
      # part e
      R = R_delta@R
    
    x_global = R[:,0]
    # print(R)
    # print(np.linalg.inv(R))
    return x_global

### Run & Test
Use the cell below to run and test Part 1.

In [38]:
def get_deviation(calculated, expected):
    calculated = np.array(calculated)
    expected = np.array(expected)
    with np.errstate(divide='ignore', invalid='ignore'):
        dot_prod = np.dot(calculated, expected) / \
            np.linalg.norm(calculated) / np.linalg.norm(expected)
        return np.degrees(np.arccos(dot_prod))


if __name__ == '__main__':
    gt_final = [-0.03729062, 0.98638029, -0.16019777]
    stu_final = track_orientation('orient_data/gyro.csv')
    output = [['Test', 'Dataset', 'Expected Output',
               'Your Output', 'Deviation', 'Result', 'Grade']]
    final_state = 'FAILED'
    final_grade = 0
    final_dev = get_deviation(stu_final, gt_final)
    if(final_dev < 2):
        final_state = 'PASSED'
        final_grade = 10
    output.append(['Final Orientation', 'gyro.csv',
                  gt_final, stu_final, "{:2.2f}°".format(final_dev), final_state, "{} / 10".format(final_grade)])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    output.append(['<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>'])
    output.append(['<i>👻 Hidden test 4 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    display_table(output)

---
## Part 2. Step Estimation
We have provided you with smartphone accelerometer data collected under three circumstances
<ol type="A">
  <li>walking with phone in pant pocket</li>
  <li>walking with phone held in the hand statically as if the user is looking at it while walking</li>
  <li>walking with phone in hand and the hand swinging</li>
</ol>

there are three columns, representing the accelerometer readings in three local axes(unit: $m / s^{2}$). The accelerometer is sampled at 100Hz.

Implement your algorithm in the `calculate_steps(accl_data)` and `calculate_final_position(steps_with_walking_direction, start_position)` functions below. Do NOT change the function signatures. You are, however, free to define and use helper functions. You are expected to use common signal processing and linear algebra functions (e.g., high/low pass filtering, convolution, cross correllation, peak detection, fft etc.) 

<b>TIP:</b> It may help to first visualize the accelerometer data like you did in MP1.

In [39]:
def dc_block(signal, window_size=20):
    """
    Applies a moving average filter of window_size to a numpy array x.
    """
    mean=np.average(signal[:window_size])
    output = np.zeros_like(signal)
    for i in range(len(signal)):
      if i >= window_size:
        mean -= signal[i - window_size] / window_size
        mean += signal[i] / window_size
      output[i] = signal[i] - mean
    return output

In [43]:
def threshold(signal, window_size=10):
    """
    The average threshold of the filtered norm.
    """
    mean=0
    output = np.zeros_like(signal)
    for i in range(len(signal)):
      if i >= window_size:
        mean -= signal[i - window_size] / window_size
      mean += signal[i] / window_size
      output[i] = mean
    return output

In [44]:
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

In [197]:
def get_step_length():
    """
    step length is approximated to be proportional to the height of pedestrian
    """
    height=1.75 # in meters
    return 0.415*height


def calculate_steps(accl_data):
    """
    :param accel_data: pandas dataframe with 3 columns, "x", "y", "z" representing acceleration in m/s^2 
    and index of dataframe is "timestamp"
    
    returns: pandas dataframe with 2 columns "timestamp" and "steplength" 
    
    for this MP assume that the step length is same for each step, 
    use the get_step_length function to get the step length
    
    the timestamp is the time when the step is detected
    
    NOTE: timestamps should be among the values in the timestamp column, 
    you are not expected to do any time interpolation
    """
    ########################################
    # Implementation details
    # Paper: Design and Implementation of Practical Step Detection Algorithm for Wrist-Worn Devices
    # Authors: Yunhoon Cho, Hyuntae Cho, and Chong-Min Kyung, Fellow, IEEE

    ### Step Counting using peak detection

    ########################################

    # Step 1: Data Preprocessing

    # Norm Calculation
    x = accl_data['x'].values
    y = accl_data['y'].values
    z = accl_data['z'].values
    norm = np.sqrt(np.square(x) + np.square(y) + np.square(z))
    
    # # comment out #########
    # print("Length of norm:", len(norm))
    # print(type(norm))
    # plt.figure(figsize=(18, 6))
    # plt.plot(norm)
    # plt.title('Norm, acceleration')
    # plt.show()
    # #########

    # DC component removal
    window_size = 20
    norm_zeroDC = dc_block(norm, window_size)

    # # comment out #########
    # print("Length of norm_zero_dc:", len(norm_zeroDC))
    # plt.figure(figsize=(18, 6))
    # plt.plot(norm_zeroDC)
    # plt.title('Norm remove DC offset, acceleration')
    # plt.show()
    # #########

    ########################################

    # Step 2: Data Filtering Phase
    
    # define the transfer function coefficients
    h = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    # apply the transfer function using convolution and scaling
    norm = np.convolve(norm_zeroDC, h, mode='same') / 100

    # norm = butter_lowpass_filter(data=norm_zeroDC, cutoff=20, fs=100, order=20)

    # # comment out #########
    # print("Low Pass Filtered Norm:", len(norm))
    # plt.figure(figsize=(18, 6))
    # plt.plot(norm)
    # plt.title('Low Pass Filtered, acceleration')
    # plt.show()
    # #########

    ########################################

    # Step 3: Peak Detection using sign-of-slope method

    # Average Threshold of filtered norm calculation with window size of 10
    thr_avg = threshold(norm, 10)

    peaks, _ = scipy.signal.find_peaks(norm, height=None, threshold=None, distance=None, prominence=0.8
                                       , width=None, wlen=None, rel_height=0.5, plateau_size=None)

    # print(peaks)
    # new_peaks = []
    # for i in peaks:
    #   if norm[i] >= thr_avg[i]:
    #     new_peaks.append(i)
    # peaks = np.array(new_peaks)
    # print(peaks)
    
    # # comment out #########
    # print("Low Pass FIltered Norm:", len(norm))
    # plt.figure(figsize=(18, 6))
    # plt.plot(norm)
    # plt.title('Low Pass Filtered, acceleration')
    # for i in peaks:
    #   plt.plot(i, norm[i], 'x', markersize=10, color='red')
    # plt.show()
    # #########



    # replace the following dummy with your implementation
    steps = pd.DataFrame({"timestamp": np.around(peaks, -1).tolist(), 
                          "steplength": get_step_length()})
    # print(type(steps['timestamp'][0]))
    
    return steps

def calculate_final_position(steps_with_walking_direction, start_position):
    """ 
    :param steps_with_walking_direction: pandas dataframe with 3 columns "timestamp", "steplength", and "walking_direction"
    
    walking_direction is an angle in degrees with global frame x-axis. It can be from 0 degrees to 360 degrees.
    for e.g. if walking direction is 90 degrees, user is walking in the positive y-axis direction
    
    NOTE: In the given test cases, the walking direction is same through out the trajectory 
    but in hidden cases the walking direction may change
    
    step_length is in meters
    
    :param start_position: starting position of the user. It is tuple (x,y)
    
    return (x,y) coordinate of the final position in meters
    """

    timestamp = steps_with_walking_direction['timestamp'].values
    steplength = steps_with_walking_direction['steplength'].values
    walking_direction = steps_with_walking_direction['walking_direction'].values

    final_position = [start_position[0], start_position[1]]

    for i in range(steps_with_walking_direction.shape[0]):
      angle = np.radians(walking_direction[i])
      c = np.cos(angle) * steplength[i]
      s = np.sin(angle) * steplength[i]
      final_position[0] += c
      final_position[1] += s




    return (final_position[0], final_position[1])

### Run & Test
Use the cell below to run and test `calculate_steps(accl_data)` and `calculate_final_position(steps_with_walking_direction, start_position)`. 

In [199]:
def estimate_steps_score(calculated, expected):
    delta = abs(calculated - expected)
    return 1 if(delta <= 2) else max((1 - abs(delta - 2) / expected), 0)

def estimate_pdr_score(calculated, expected):
    error = math.sqrt((calculated[0] - expected[0])**2 + (calculated[1] - expected[1])**2)
    # error more than 5 meters, score is 0
    return 1.0 if (error <= 2.0) else max(0, 1-(error-2.0)/3)


if __name__ == '__main__':
    data_dir = "accel_data"
    walk_accl_files = ['holdstatic_20steps.csv', 'inpocket_26steps.csv',
                       'inpocket_36steps.csv', 'swing_32steps.csv', 'swing_38steps.csv']
    groundtruth_numsteps = [20, 26, 36, 32, 38]
    groundtruth_final_positions = [(12.58, 7.26), (18.88, 0.0), (0.0, 26.14), (-23.24, 0.0), (0.0, -27.6)]
    output = [['Dataset', 'Expected Number Steps', 'Your Number of Steps', 'Expected Final Position', 'Your Final Position', 'Grade']]
    for i in range(len(groundtruth_numsteps)):
        accel_data = pd.read_csv(str(Path(data_dir) / walk_accl_files[i]))
        accel_data = accel_data.set_index("timestamp", drop=True)
        calculated = calculate_steps(accel_data)
        assert len(calculated.columns)==2
        assert "timestamp" in calculated.columns
        assert "steplength" in calculated.columns
        calculated_num_steps = calculated.shape[0]
        calculated["walking_direction"] = accel_data["walking_direction"][calculated["timestamp"]].values
        calculated_final_position = calculate_final_position(calculated, (0.0,0.0))
        steps_score = estimate_steps_score(calculated_num_steps, groundtruth_numsteps[i])
        pdr_score = estimate_pdr_score(calculated_final_position, groundtruth_final_positions[i])
        # final score is weighted sum of both scores
        score = 0.7*steps_score + 0.3*pdr_score
        output.append([walk_accl_files[i], groundtruth_numsteps[i],
                      calculated_num_steps, groundtruth_final_positions[i], calculated_final_position,  "{:2.2f} / 5.00".format(score * 5)])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 15.00'])
    output.append(['<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>'])
    output.append(['<i>👻 Hidden test 5 👻</i>','<i>???</i>', '<i>???</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 15.00'])
    display_table(output)

---
## Rubric

####  Orientation Tracking (100 points) 
You will be graded on the provided data as well as 4 addition sets of data. They are each worth 20 points. A 2-degree error is tolerated. For results greater than the error threshold, no points will be rewarded since we provided a detailed algorithm to follow. The test data also include the simple case where the phone will only rotate along Z axis onwards. (In case you find the MP too difficult, only doing 1D integration on Z axis should at least give you some points.)

#### Step Estimation (100 points) 
You will be graded on the 5 sets of provided data (5 points each), as well as 5 sets of hidden data (15 points each). For each test case, the grade depends on how much the result deviates from the groudtruth. A 2-step error for the provided data is tolerated. A 4-step error for the hidden data is tolerated. For results greater than the error threshold, your score will be scaled proportionally. The score is the weighted sum of PDR error i.e., deviation from the final position and the step count detection accuracy.

---
## Submission Guideline
This Jupyter notebook is the only file you need to submit on Gradescope. Make sure to fill your name at the top of this file.

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**